In [9]:
# ================================================================
# 📦 KOD BLOĞU 1: Kütüphaneler, Ayarlar ve Bağlantı (Hugging Face Uyumlu)
# ================================================================

import os
import re
import shutil
import json
from pathlib import Path
from typing import List, Dict
import pandas as pd
from datasets import load_dataset # Hugging Face datasets kütüphanesi gerekli olabilir

from dotenv import load_dotenv, find_dotenv
import google.generativeai as genai
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# --- HUGGING FACE AYARLARI ---
DATASET_NAME = "Renicames/turkish-law-chatbot"
SPLIT_NAME = "train" 
# Veri seti eğer zaten chunk'lanmışsa, CHUNK_SIZE/OVERLAP devre dışı kalır.
# Eğer parçalama gerekiyorsa, bunu KOD BLOK 2'de ayarlarız.

# --- CHROMA DB AYARLARI (Mini-Dataset yolu aynı kalır) ---
CHROMA_PATH = os.path.expanduser("~/chroma_db_lexmove_mini") 
COLLECTION_NAME = "mevzuat_chunks_mini"
EMBEDDING_MODEL = "all-MiniLM-L6-v2"

# --- LLM VE API KEY SETUP ---
load_dotenv(find_dotenv(usecwd=True))
api_key = os.getenv('GOOGLE_API_KEY')
if api_key:
    genai.configure(api_key=api_key)
    print("✅ API Anahtarı yüklendi.")

# Embedding Model
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print("✅ Embedding Modeli yüklendi.")


✅ API Anahtarı yüklendi.
✅ Embedding Modeli yüklendi.


In [13]:
# ================================================================
# 📦 KOD BLOĞU 2: Hugging Face Q&A Veri Yükleme ve Document Oluşturma 
# ================================================================

# Veri setindeki GERÇEK sütun isimleri
SORU_COL = 'Soru' 
CEVAP_COL = 'Cevap' 

# Not: Bu veri seti zaten Q&A formatında olduğu için Text Splitter kullanmıyoruz.

def load_and_create_documents() -> List[Document]:
    """
    Hugging Face Q&A veri setini yükler ve her Cevabı, Soru metadatası ile 
    LangChain Document listesine dönüştürür.
    """
    all_documents = []
    
    print(f"⏳ Hugging Face'ten veri seti yükleniyor: {DATASET_NAME} ({SPLIT_NAME})")
    
    try:
        # Hugging Face'ten doğrudan pandas DataFrame olarak yükleme
        dataset = load_dataset(DATASET_NAME, split=SPLIT_NAME)
        df = dataset.to_pandas()
        
    except Exception as e:
        print(f"❌ Veri seti yüklenirken HATA oluştu: {e}")
        return []
        
    print(f"✅ Veri seti yüklendi. Toplam satır (Document adayı): {len(df)}")
    
    # DataFrame satırlarını LangChain Document'a dönüştürme
    for index, row in df.iterrows():
        try:
            # Sütun isimlerini kullanarak verileri çek
            question = str(row[SORU_COL]).strip()
            answer = str(row[CEVAP_COL]).strip()
            
            # İçerik olarak CEVAP'ı kullanıyoruz
            page_content = answer 
            
            # Soru veya Cevap çok kısa ise atla
            if not page_content or len(page_content) < 20:
                continue
                
            metadata = {
                "source": DATASET_NAME,         
                "question": question,           
                "kanun_adi": "ÇEŞİTLİ MEVZUAT", 
                "madde_no": f"Soru-{index}"     
            }
            
            document = Document(
                page_content=page_content,
                metadata=metadata
            )
            all_documents.append(document)
            
        except KeyError:
            print(f"⚠️ Hata: Veri setinde '{SORU_COL}' veya '{CEVAP_COL}' sütunları bulunamadı. Lütfen sütun isimlerini kontrol edin.")
            return []
        except Exception as e:
            print(f"❌ Satır {index} işlenirken hata: {e}")
            continue
            
    print(f"\n📚 TOPLAM {len(all_documents)} Document nesnesi hazırlandı.")
    return all_documents

# Tüm veriyi yükle ve Document listesini oluştur
documents = load_and_create_documents()

⏳ Hugging Face'ten veri seti yükleniyor: Renicames/turkish-law-chatbot (train)
✅ Veri seti yüklendi. Toplam satır (Document adayı): 13354

📚 TOPLAM 13170 Document nesnesi hazırlandı.


In [15]:
# ================================================================
# 📦 KOD BLOĞU 3: ChromaDB Oluşturma/Güncelleme
# ================================================================

def create_or_update_chromadb(docs: List[Document]):
    """Yeni verileri ChromaDB'ye yazar ve eski veriyi siler."""
    
    # Eski ChromaDB'yi silme (Temiz bir başlangıç için şiddetle tavsiye edilir)
    if Path(CHROMA_PATH).exists():
        print(f"⚠️ Eski veritabanı siliniyor: {CHROMA_PATH}")
        shutil.rmtree(CHROMA_PATH)
    
    if not docs:
        print("❌ Veritabanına yazılacak doküman yok. İşlem iptal edildi.")
        return None

    print(f"⏳ Veritabanı oluşturuluyor ve {len(docs)} parça yazılıyor...")
    
    # ChromaDB'ye yazma
    db = Chroma.from_documents(
        docs, 
        embeddings, 
        persist_directory=CHROMA_PATH, 
        collection_name=COLLECTION_NAME
    )
    
    db.persist()
    print("✅ ChromaDB başarıyla oluşturuldu/güncellendi.")
    print(f"📊 Toplam parça sayısı: {db._collection.count()}")
    return db

# ChromaDB'yi oluştur
mini_vectorstore = create_or_update_chromadb(documents)


⏳ Veritabanı oluşturuluyor ve 13170 parça yazılıyor...
✅ ChromaDB başarıyla oluşturuldu/güncellendi.
📊 Toplam parça sayısı: 13170


/var/folders/yt/j0nqv5qs2yv0qzc800s3hgfh0000gn/T/ipykernel_27371/4059546402.py:27: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()
